In [25]:
import numpy as np

In [26]:
data=np.load("part1b.npz")

In [27]:
data.files

['N', 'Lc', 'Ic_0', 'gamma', 'L_validation', 'L_test', 'betas_validation']

In [28]:
data["N"]

array(50000.)

In [29]:
data['Lc'].shape

(4,)

In [30]:
nx=np.sum(data['Ic_0'],axis=1)

In [31]:
data["L_validation"].shape

(20, 20)

In [32]:
L_val.shape

(20, 20)

In [54]:
N=50000
Ic_01=data['Ic_0']
Ic_0=nx
not_inf=50000-Ic_0 #origin population not infected 
nd=not_inf.reshape(-1,1)
print(not_inf)
print(leaf1.shape)
print(leaf1)


leaf_nodes=np.array([[.5,.3,.1,.1],[.4,.3,.2,.1],[.3,.3,.2,.2],[.1,.2,.3,.4]])
leafs=leaf_nodes.reshape(-1,1)

leaf1=np.dot(nd,leafs.transpose())# finding the intial suspectiable population 
#leaf2=np.dot(leaf_nodes,not_inf.transpose())
#leaf1



L_val=data["L_validation"]
Lc=data['Lc']
#lc1=Lc.reshape(-1,1)
betas_v=data["betas_validation"]

Rc=np.array([0])
t=np.linspace(0,20,20)
I=np.sum(Ic_0)
Gamma=data["gamma"]


[49747 49737 49818 49800 49832 49845 49776 49804 49740 49899 49835 49741
 49774 49811 49792 49897 49791 49659 49808 49840]
(20, 16)
[[24873.5 14924.1  4974.7  4974.7 19898.8 14924.1  9949.4  4974.7 14924.1
  14924.1  9949.4  9949.4  4974.7  9949.4 14924.1 19898.8]
 [24868.5 14921.1  4973.7  4973.7 19894.8 14921.1  9947.4  4973.7 14921.1
  14921.1  9947.4  9947.4  4973.7  9947.4 14921.1 19894.8]
 [24909.  14945.4  4981.8  4981.8 19927.2 14945.4  9963.6  4981.8 14945.4
  14945.4  9963.6  9963.6  4981.8  9963.6 14945.4 19927.2]
 [24900.  14940.   4980.   4980.  19920.  14940.   9960.   4980.  14940.
  14940.   9960.   9960.   4980.   9960.  14940.  19920. ]
 [24916.  14949.6  4983.2  4983.2 19932.8 14949.6  9966.4  4983.2 14949.6
  14949.6  9966.4  9966.4  4983.2  9966.4 14949.6 19932.8]
 [24922.5 14953.5  4984.5  4984.5 19938.  14953.5  9969.   4984.5 14953.5
  14953.5  9969.   9969.   4984.5  9969.  14953.5 19938. ]
 [24888.  14932.8  4977.6  4977.6 19910.4 14932.8  9955.2  4977.6 14932

In [55]:
def SIR(t,y,betas,gammas):
    S_vec=y[0:16]
    I_vec=y[16:20]
    R_vec=y[20]
    
    N=50000 ##population size 
    
    dsvc_dt=-(betas*S_vec*sum(I_vec)/N)
    dic_dt=-(np.sum(dsvc_dt,axis=0))-(gammas*I_vec)
    dr_dt=gammas*sum(I_vec)
    
    vectors=np.concatenate([dsvc_dt.flatten(),dic_dt.flatten(),dr_dt.flatten()])
    #breakpoint()
    return vectors

In [57]:
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp
beta_grid=np.linspace(0.01,1.0,20)
lf_nodes=20
observed_data=L_val
mmse=np.inf
beta_mmse=np.nan
Rc=np.zeros((lf_nodes))
for (beta) in beta_grid:
    cumulative_mse = 0.0
    
    for node_index in range(lf_nodes):
        initial_conditions=np.concatenate([leaf1[node_index].flatten(), Ic_01[node_index].flatten(), Rc[node_index].flatten()])

        soln_obj=solve_ivp(SIR, (t[0], t[-1]), initial_conditions,\
                                     t_eval=t, args=(beta, Gamma))


        S_predicted=soln_obj.y[0:16]
        I_predicted=soln_obj.y[16:20]
        R_predicted=soln_obj.y[20]
        
        
        L_predicted=np.dot(I_predicted.reshape(-1,1),(Lc.reshape(-1,1)).transpose())
        dd=np.sum(L_predicted,axis=0)
        
        
        cumulative_mse += mean_squared_error(observed_data[node_index], dd) 
    
    if cumulative_mse<mmse:
        mmse = mse
        beta_mmse=beta
        gamma_mmse=gamma
        
print(f"True values are beta={beta_true:.2f}, gamma={gamma_true:.2f}")
print(f"Estimated values are beta={beta_mmse:.2f}, gamma={gamma_mmse:.2f}")


> <ipython-input-55-9ff97b678b34>(14)SIR()
     10     dr_dt=gammas*sum(I_vec)
     11 
     12     vectors=np.concatenate([dsvc_dt.flatten(),dic_dt.flatten(),dr_dt.flatten()])
     13     breakpoint()
---> 14     return vectors

ipdb> S_vec
array([24873.5, 14924.1,  4974.7,  4974.7, 19898.8, 14924.1,  9949.4,
        4974.7, 14924.1, 14924.1,  9949.4,  9949.4,  4974.7,  9949.4,
       14924.1, 19898.8])
ipdb> vectors
array([-1.2585991 , -0.75515946, -0.25171982, -0.25171982, -1.00687928,
       -0.75515946, -0.50343964, -0.25171982, -0.75515946, -0.75515946,
       -0.50343964, -0.50343964, -0.25171982, -0.50343964, -0.75515946,
       -1.00687928,  6.71164994,  3.64022137,  6.42593566,  5.42593566,
       18.07142857])
ipdb> q


BdbQuit: 